In [2]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from numpy.linalg import norm
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer
elastic_rods.set_max_num_tbb_threads(4)

def centralJoint(l):
    positions = [j.position for j in l.joints()]
    center = np.mean(positions)
    return np.argmin([norm(p - center) for p in positions])


l = elastic_rods.RodLinkage('../../sweep/181101_Randomish/20181106_161318_meshID_fe45c5cc-ce5d-423c-80e0-34a089bc6a7d.obj', 20)
mat = elastic_rods.RodMaterial('+', 2000, 0.3, [2.0, 2.0, 0.2, 0.2])
l.setMaterial(mat)
driver = centralJoint(l)

with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024)
view.setCameraParams(((-2.200522340164312, 0.5782056167253651, -0.7549139983294457),
 (-0.02031290489160542, -0.03507141600564204, -0.999178353285445),
 (-0.17091934654983643, 0.17173250980468135, 0.09235104577312339)))
view.show()

Renderer(camera=PerspectiveCamera(aspect=2.0, children=(DirectionalLight(color='white', intensity=0.6, positio…

In [3]:
closed_linkage = elastic_rods.RodLinkage(l)
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-6
    opts.useIdentityMetric = True
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
open_linkage(l, driver, np.pi/12, 15, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=True, useTargetAngleConstraint=True);

target angle:  0.3858934154439101
0	12.8824	7.25672	7.25672	1	0
1	12.5803	0.0111094	0.0111094	0.03125	0
2	12.5802	0.091585	0.091585	0.5	0
3	12.5799	0.134098	0.134098	1	0
4	12.5795	0.101225	0.101225	1	0
5	12.5793	0.172864	0.172864	1	0
6	12.579	0.0568941	0.0568941	0.5	0
7	12.5789	0.150672	0.150672	1	0
8	12.5786	0.058853	0.058853	0.5	0
9	12.5785	0.12105	0.12105	1	0
10	12.5783	0.0755804	0.0755804	1	0
11	12.5782	0.191079	0.191079	1	0
12	12.5779	0.0294044	0.0294044	0.25	0
13	12.5778	0.0922227	0.0922227	1	0
14	12.5777	0.10234	0.10234	1	0
15	12.5775	0.0810397	0.0810397	1	0
16	12.5774	0.118461	0.118461	1	0
17	12.5773	0.0563396	0.0563396	1	0
18	12.5772	0.220619	0.220619	1	0
19	12.577	0.0162615	0.0162615	0.25	0
20	12.577	0.141013	0.141013	0.25	0
0.16901845125121712	12.576961783330795	12.576961783330793	12.281273740896346	0.21659412641783882	0.07909391601660115
target angle:  0.4033467079638534
0	12.9965	7.79424	7.79424	1	0
1	12.644	0.0115906	0.0115906	0.0625	0
2	12.6438	0.13614	0.13614	1	0
3	12.6

In [4]:
elastic_rods.linkage_deformation_analysis(closed_linkage, l, 'test_defo_analysis.msh')

In [5]:
l.writeLinkageDebugData('test.msh')